In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import requests
import zipfile
import pandas as pd
import os
import pickle

token = os.environ['EPC_TOKEN']

In [ ]:
import requests
import os
import pickle
# import configs

epc_file_path = "../../../../data/epc-files.pkl"
with open(epc_file_path, "rb") as f:
    epc_files = pickle.load(f)
    
token = os.environ['EPC_TOKEN']

def download_epc_file(local_folder, file, chunk_size):

    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    output_file = f"{local_folder}/{file}"
    url = f"https://epc.opendatacommunities.org/api/v1/files/{file}"
    headers = {
        "Authorization": f"Basic {token}"
    }
    
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
    
        with open(output_file, "wb") as out_file:
            for chunk in response.iter_content(chunk_size):
                out_file.write(chunk)
        print(f"{file} downloaded.")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")


def download_epc_file_list(local_folder, file_list, chunk_size):
    for file in file_list:
        download_epc_file(local_folder, file, chunk_size)

In [ ]:
# epc_files = epc_files[:3]
# epc_files

In [ ]:
local_folder = 'prova_epc/'
chunk_size = 100_000

download_epc_file_list(local_folder, epc_files, chunk_size)

In [ ]:
import zipfile
import os

def extract_file(local_folder, file):
    zipped_file = f"{local_folder}{file}"
    extract_to = f"{zipped_file}"[:-4]
    try:
        with zipfile.ZipFile(zipped_file, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
    except Exception as e:
        print(f"Error: {e}")

def extract_file_list(local_folder, file_list):
    for file in file_list:
        extract_file(local_folder, file)
        os.remove(f"{local_folder}{file}")

In [ ]:
extract_to = f"{zipped_file}"[:-4]

extract_to

In [ ]:
!rm -rf prova

In [62]:
import pandas as pd
from bucket_utils import Bucket
import pickle
import os

   
london_local_authorities_path = "../../../../data/london-local-authorities.csv"
london_authorities = list(
    pd.read_csv(london_local_authorities_path)\
    ['Code']
)

# epc_columns_path = "../../../../data/epc-columns.pkl"
# with open(epc_columns_path, "rb") as f:
#     columns = pickle.load(f)
# columns_set = set(columns) 
columns = ["ADDRESS","POSTCODE","TOTAL_FLOOR_AREA"]
columns_set = set(columns)

def transform(df):
    df = df[df['LOCAL_AUTHORITY'].isin(london_authorities)]
    cols_in_file_set = set(df.columns)
    missing_cols = columns_set - cols_in_file_set
    for col in missing_cols:
        df.loc[:,col] = None
    for col in columns:
        df = df[df[col].notnull()]
    for col in columns:
        df[col] = df[col].astype(str)
    df = df[columns]
    return df
    

def stream_file_to_bucket(bucket_session,
                          local_folder, 
                          local_filename,
                          destination_folder,
                          chunk_size):

    df_iter = pd.read_csv(f"{local_folder}{local_filename}", iterator=True, chunksize=chunk_size,low_memory=False)

    for i, chunk in enumerate(df_iter):
        chunk_file_name = f"{local_filename[:-4]}_{i}.parquet"
        local_file_path = f"{local_folder}{chunk_file_name}"
        chunk = transform(chunk)
        if len(chunk)>0:
            chunk.to_parquet(local_file_path)
            with open(local_file_path, 'rb') as file:
                bucket_session.put_object(local_folder, chunk_file_name, 
                                          destination_folder, chunk_file_name)
            os.remove(local_file_path)

def stream_epc_files_to_bucket(file_list, 
                               region, 
                               namespace, 
                               bucket_name,
                               epc_local_folder,
                               epc_bucket_folder,
                               chunk_size):

    bucket_session = Bucket(region, namespace, bucket_name)
    
    for file in file_list:
        file = file[:-4]
        epc_unzipped_file_folder = file+"/"
        local_folder = f"{epc_local_folder}{epc_unzipped_file_folder}"
        stream_file_to_bucket(bucket_session = bucket_session,
                              local_folder = local_folder, 
                              local_filename = 'certificates.csv',
                              destination_folder = f"{epc_bucket_folder}{local_folder.split('/')[1]}/",
                              chunk_size=chunk_size)

    bucket_session.close()

In [435]:
"domestic-2023.zip"[:-4]

'domestic-2023'

In [ ]:
No such file or directory: 'epc-download-chunks/domestic-2023.zip/certificates.csv'

In [63]:
file_list = ['display-2024-03', 'domestic-2021','domestic-2023']

OCI_REGION ='uk-london-1'
OCI_NAMESPACE = 'lrqgbz9z6zlj'
bucket_name = 'london-property-sales-price'
epc_local_folder = "prova/"
epc_bucket_folder = "epc-download-chunks/"
chunk_size = 100_000

stream_epc_files_to_bucket(file_list, 
                            OCI_REGION, 
                            OCI_NAMESPACE, 
                            bucket_name,
                            epc_local_folder,
                            epc_bucket_folder,
                            chunk_size)

In [ ]:
# stream_file_to_bucket(bucket_session = bucket_session,
#                       local_folder = local_folder, 
#                       local_filename = 'certificates.csv',
#                       destination_folder = f"{epc_bucket_folder}{local_folder.split('/')[1]}/",
#                       chunk_size=chunk_size)

In [481]:
from bucket_utils import Bucket

OCI_REGION ='uk-london-1'
OCI_NAMESPACE = 'lrqgbz9z6zlj'
bucket_name = 'london-property-sales-price'

bucket_session = Bucket(OCI_REGION, OCI_NAMESPACE, bucket_name)

In [487]:
bucket_session.list_objects()

['epc-download-chunks/display-2023/certificates_0.parquet',
 'epc-download-chunks/domestic-2023/certificates_0.parquet',
 'epc-download-chunks/domestic-2023/certificates_1.parquet',
 'epc-download-chunks/domestic-2023/certificates_10.parquet',
 'epc-download-chunks/domestic-2023/certificates_11.parquet',
 'epc-download-chunks/domestic-2023/certificates_12.parquet',
 'epc-download-chunks/domestic-2023/certificates_13.parquet',
 'epc-download-chunks/domestic-2023/certificates_14.parquet',
 'epc-download-chunks/domestic-2023/certificates_15.parquet',
 'epc-download-chunks/domestic-2023/certificates_16.parquet',
 'epc-download-chunks/domestic-2023/certificates_2.parquet',
 'epc-download-chunks/domestic-2023/certificates_3.parquet',
 'epc-download-chunks/domestic-2023/certificates_4.parquet',
 'epc-download-chunks/domestic-2023/certificates_5.parquet',
 'epc-download-chunks/domestic-2023/certificates_6.parquet',
 'epc-download-chunks/domestic-2023/certificates_7.parquet',
 'epc-download-chu

In [61]:
bucket_session.delete_folder_content("epc-download-chunks")

All objects in the folder 'epc-download-chunks' have been deleted.


In [ ]:
cols_epc = ["ADDRESS","POSTCODE","TOTAL_FLOOR_AREA"]

## Merging PP data with EPC data
# The two datasets are merged by simply using the address fields, which is formatted in both datasets before being processed.

def transform(df, cols:list):
    transformed_cols = []
    for c in cols:
        newc = c+"_fmt"
        df[newc] = df[c].str.strip()
        df[newc] = df[newc].str.lower()
        df[newc] = df[newc].str.replace(" ","")
        df[newc] = df[newc].str.replace(",","")
        transformed_cols.append(newc)
    return transformed_cols

pp_cols = ["SAON", "PAON", "STREET","POSTCODE"]

transformed_cols_pp = transform(pp_london, pp_cols)

pp_london["address_match_string"] = pp_london[transformed_cols_pp].fillna("").astype(str).agg(lambda x:''.join(x), axis=1)


epc_cols = ['ADDRESS', 'POSTCODE']
transformed_cols_epc = transform(london_epc, epc_cols)

london_epc["address_match_string"] = london_epc[transformed_cols_epc].fillna("").astype(str).agg(lambda x:''.join(x), axis=1)

In [488]:
#### MERGE PPD TO EPC


### variables

region ='uk-london-1'
namespace = 'lrqgbz9z6zlj'
bucket_name = 'london-property-sales-price'

ppd_download_bucket_folder = 'ppd-download-chunks/'
epc_download_bucket_folder = 'epc-download-chunks/'
save_as_table_name = "london"
write_mode = "overwrite"

In [492]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
from pyspark.sql import types
from bucket_utils import Bucket
from datetime import datetime


def bucket_to_dwh(region,
                  namespace, 
                  bucket_name,
                  ppd_download_bucket_folder,
                  epc_download_bucket_folder,
                  save_as_table_name,
                  write_mode):

    match_rate_path = "../../../../data/match_rate_log.txt"
    
    OCI_ACCESS_KEY_ID = os.environ['OCI_ACCESS_KEY_ID']
    OCI_SECRET_ACCESS_KEY = os.environ['OCI_SECRET_ACCESS_KEY']
    OCI_REGION = region
    OCI_NAMESPACE = namespace
    BUCKET_NAME = bucket_name
    
    db_url = "jdbc:postgresql://pgwarehouse:5432/london"
    db_properties = {
        "user": "root",
        "password": "root",
        "driver": "org.postgresql.Driver"
    }
    
    spark = SparkSession.builder \
        .master("local[*]") \
        .appName('Transform') \
        .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.506') \
        .config('spark.hadoop.fs.s3a.endpoint', f'https://{OCI_NAMESPACE}.compat.objectstorage.{OCI_REGION}.oraclecloud.com') \
        .config('spark.hadoop.fs.s3a.access.key', OCI_ACCESS_KEY_ID) \
        .config('spark.hadoop.fs.s3a.secret.key', OCI_SECRET_ACCESS_KEY) \
        .config('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
        .config('spark.hadoop.fs.s3a.path.style.access', 'true') \
        .config('spark.hadoop.fs.s3a.connection.ssl.enabled', 'true') \
        .config("spark.jars", "/opt/spark/jars/postgresql-42.7.4.jar") \
        .getOrCreate()


    ppd_path = f's3a://{BUCKET_NAME}/{ppd_download_bucket_folder}/*.parquet'
    ppd = spark.read.parquet(ppd_path, header=True, inferSchema=True)
    
    bucket_session = Bucket(OCI_REGION, OCI_NAMESPACE, bucket_name)
    objects = bucket_session.list_objects()
    
    folders = [x for x in objects if epc_download_bucket_folder in x]
    
    folders = set([x.split("/")[1] for x in folders])
    
    epc_path = f"s3a://{BUCKET_NAME}/"
    
    epc_file_paths = [f"{epc_path}/{epc_download_bucket_folder}/{folder}/*.parquet" for folder in folders]
    
    epc = spark.read.parquet(*epc_file_paths)
    
    epc = epc.withColumn("TOTAL_FLOOR_AREA", F.col("TOTAL_FLOOR_AREA").cast(types.IntegerType()))
    epc = epc.na.fill({"TOTAL_FLOOR_AREA": 0})
    
    def create_key(df, column_list):
        new_cols = []
        for col in column_list:
            new_col = col+"_fmt"
            df = df.withColumn(new_col, F.nvl(col, F.lit("")))
            df = df.withColumn(new_col, F.trim(new_col))
            df = df.withColumn(new_col, F.lower(new_col))
            df = df.withColumn(new_col, F.regexp_replace(new_col," ", ""))
            df = df.withColumn(new_col, F.regexp_replace(new_col,",", ""))
            new_cols.append(new_col)
        df = df.withColumn("KEY", F.concat(*new_cols))
        df = df.na.fill({"KEY": ""})
        df = df.drop(*new_cols)
        return df
    
    ppd_cols_keys = ["SAON", "PAON", "STREET","POSTCODE"]
    ppd = create_key(ppd, ppd_cols_keys)
    
    epc_cols_keys = ['ADDRESS', 'POSTCODE']
    epc = create_key(epc, epc_cols_keys)
    
    # DEDUPE EPC
    # For addresses that have multiple values for "TOTAL_FLOOR_AREA", keep only the highest.
    max_floor_area = epc.groupby('key').agg(F.max("TOTAL_FLOOR_AREA").alias("TOTAL_FLOOR_AREA"))
    epc = epc.join(max_floor_area, on=['key', 'TOTAL_FLOOR_AREA'])[['key', 'TOTAL_FLOOR_AREA']].distinct()
    
    data = ppd.join(epc, how='left', on='key')
    
    match_agg = data.groupby(data['TOTAL_FLOOR_AREA'].isNotNull().alias("is_match")).count()
    cnt_match = match_agg[match_agg["is_match"]==True].collect()[0][1]
    cnt_tot = data.count()
    match_rate = cnt_match/cnt_tot
    
    timestamp = str(datetime.today()).split(".")[0]
    with open(match_rate_path, "a") as f:
        f.write(f"\n{timestamp}|{match_rate:.2f}")
    
    data = data.withColumn("PARTIAL_POSTCODE", F.split_part(F.col('postcode'), F.lit(' '), F.lit(1)))
    
    data = data.withColumn(
        "PROPERTY_TYPE",
        F.when(data["PROPERTY_TYPE"] == "D", "Detached")
         .when(data["PROPERTY_TYPE"] == "S", "Semi-Detached")
         .when(data["PROPERTY_TYPE"] == "T", "Terraced")
         .when(data["PROPERTY_TYPE"] == "F", "Flats/Maisonettes")
         .when(data["PROPERTY_TYPE"] == "O", "Other")
         .otherwise(data["PROPERTY_TYPE"])
    )
    
    data = data.withColumn(
        "OLD_NEW",
        F.when(data["OLD_NEW"] == "Y", "newly built")
         .when(data["OLD_NEW"] == "N", "old")
         .otherwise(data["OLD_NEW"])
    )
    
    data = data.withColumn(
        "DURATION",
        F.when(data["DURATION"] == "F", "Freehold")
         .when(data["DURATION"] == "L", "Leasehold")
         .otherwise(data["DURATION"])
    )
    
    data = data.drop('key')
    
    data = data.na.fill({"TOTAL_FLOOR_AREA": 0})
    
    data = data.withColumn('DATE_OF_TRANSFER',F.col('DATE_OF_TRANSFER').cast(types.DateType()))
    
    data.write.jdbc(url=db_url, table=save_as_table_name, mode=write_mode, properties=db_properties)
    
    spark.stop()

In [493]:
bucket_to_dwh(region,
                  namespace, 
                  bucket_name,
                  ppd_download_bucket_folder,
                  epc_download_bucket_folder,
                  save_as_table_name,
                  write_mode)

In [430]:
table_name = "london"
local_save_path = "../../../../data/output/london-sales-price.csv"

In [461]:
os.path.dirname(local_save_path)

'../../../../data/output'

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


def save_csv(table_name, local_save_path):
    db_url = "jdbc:postgresql://pgwarehouse:5432/london"
    db_properties = {
        "user": "root",
        "password": "root",
        "driver": "org.postgresql.Driver"
    }
    
    spark = SparkSession.builder \
        .master("local[*]") \
        .appName('ReadAndSave') \
        .config("spark.jars", "/opt/spark/jars/postgresql-42.7.4.jar") \
        .getOrCreate()
    
    data = spark.read.jdbc(url=db_url, table=table_name, properties=db_properties)
    
    data.coalesce(1).write.mode("overwrite").csv(local_save_path, header=True)
    
    spark.stop()

In [475]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


# def save_csv(table_name, local_save_path):
db_url = "jdbc:postgresql://pgwarehouse:5432/london"
db_properties = {
    "user": "root",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('ReadAndSave') \
    .config("spark.jars", "/opt/spark/jars/postgresql-42.7.4.jar") \
    .getOrCreate()

data = spark.read.jdbc(url=db_url, table=table_name, properties=db_properties)



In [468]:
folder = os.path.dirname(local_save_path)
if not os.path.exists(folder):
    os.makedirs(folder, mode=0o777)

In [467]:
local_save_path

'../../../../data/output/london-sales-price.csv'

In [476]:
data.toPandas().to_csv(local_save_path,index=None)

In [474]:
spark.stop()

In [438]:
import subprocess

subprocess.run(['rm', '-r', 'ciao'], capture_output=True, text=True)

# Print the output
# print(result.stdout)


In [443]:
!mkdir fol

In [480]:
!rm -rf prova

In [444]:
! echo ok > ciao1/cia/f2

In [ ]:
!